# Beamline 8.3.2 TomoPy Reconstruction

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import print_function
import numpy as np
import os
os.environ["NUMEXPR_MAX_THREADS"] = '999'
import numbers
import dxchange
import pickle
from IPython.display import display
import ipywidgets as widgets
import json
import base64
import requests
from authlib.integrations.requests_client import OAuth2Session
from authlib.oauth2.rfc7523 import PrivateKeyJWT
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from reconstructionGPU import recon_setup, recon
import time
import utils

olefile module not found


ModuleNotFoundError: No module named 'authlib'

### The following cell is just to double check that the number of threads is high enough to not throw warnings later. If for some reason it is still showing that the number of threads is 64, expect warnings.

In [ ]:
if 'NUMEXPR_MAX_THREADS' in os.environ: os.environ.pop('NUMEXPR_MAX_THREADS')
import numexpr
print('NumExpr.nthreads = ' + str(numexpr.nthreads))

### These are functions needed for job submission to NERSC. Do not alter.

### Change the inputPath and outputPath as needed. Uncomment the remainder to view files in the input directory

You can use the `listdir` functions to query `h5` files in the input directory, to select files, or you can change the filename by hand.

In [ ]:
inputPath = "/global/homes/l/lgupta/m1759/TOMODATA/"
subdirectoryname = ''

outputPath = os.path.join(inputPath,"out")
pickledparamsfile = f'{subdirectoryname}.pkl'

# filenamelist = os.listdir(inputPath)
# filenamelist.sort()
# # uncomment this to print all files, instead of just last 10
# # for i in range(0,len(filenamelist)):
# #     print(f'{i}: {filenamelist[i]}')

# filenamelist = os.listdir(inputPath)
# filenamelist.sort()
# for i in range(np.maximum(len(filenamelist)-10000,0),len(filenamelist)):
#     print(f'{i}: {filenamelist[i]}')

filename = "20210628_184523_griebenow_084626_repeat_4x.h5"

### View some important information

In [ ]:
numslices = int(dxchange.read_hdf5(os.path.join(inputPath, filename), "/measurement/instrument/detector/dimension_y")[0])
numrays = int(dxchange.read_hdf5(os.path.join(inputPath, filename), "/measurement/instrument/detector/dimension_x")[0])
pxsize = dxchange.read_hdf5(os.path.join(inputPath, filename), "/measurement/instrument/detector/pixel_size")[0] / 10.0  # /10 to convert units from mm to cm
numangles = int(dxchange.read_hdf5(os.path.join(inputPath, filename), "/process/acquisition/rotation/num_angles")[0])
propagation_dist = dxchange.read_hdf5(os.path.join(inputPath, filename), "/measurement/instrument/camera_motor_stack/setup/camera_distance")[0]
kev = dxchange.read_hdf5(os.path.join(inputPath, filename), "/measurement/instrument/monochromator/energy")[0] / 1000
angularrange = dxchange.read_hdf5(os.path.join(inputPath, filename), "/process/acquisition/rotation/range")[0]

print(f'{filename}: \nslices: {numslices}, rays: {numrays}, angles: {numangles}, angularrange: {angularrange}, \npxsize: {pxsize*10000} um, distance: {propagation_dist} mm. energy: {kev} keV')
if kev>100:
    print('white light mode detected; energy is set to 30 kev for the phase retrieval function')

### Edit parameters for reconstruction. Open the reconstruction.py file to see comments on options.

In [ ]:
functioninput = {
    "filename": filename, "inputPath": inputPath, "outputPath": outputPath, "filetype": 'dxfile', "timepoint": 0,
    "sinoused": (500,510,1),#(-1,1,1), #(start, stop, step) for which slices to recon. (-1,1,1) reconstructs just the middle slice
    "cor": None, # None, #set something other than None if you want to automatically detect center of rotation
    "corFunction": 'pc',
    "writereconstruction": 0, "writenormalized": 0, "dorecon": 1,
    "doOutliers1D": 1,
    "doFWringremoval": 0,
    "doPhaseRetrieval": 1, "alphaReg": 0.02,
    "castTo8bit": 1, "cast8bit_min": -10, "cast8bit_max": 15,
    "doPolarRing": 0, "Rarc": 20, "Rmaxwidth": 30, "Rtmax": 101, "Rthr": 101, "Rtmin": -100,
    "chunk_proj": 50, "chunk_sino": 50,
    "verbose_printing": 1,
    "angle_offset": 0,
}
recon_dictionary, cor_tomo = recon_setup(**functioninput)
print(f"center of rotation: {recon_dictionary['cor']}")

recon_dictionary['kev'] = 21.999772731
recon_dictionary['propagation_dist'] = 40.955192

### Manually set center of rotation based on overlay of first and last image (assumes 180 degree rotation)

In [ ]:
%matplotlib widget

if isinstance(cor_tomo,np.ndarray):  
    firstimage = np.squeeze(cor_tomo[-1,:,:])
    lastimageflipped = np.squeeze(cor_tomo[1,:,::-1])
    shiftedlastimage = np.zeros_like(lastimageflipped)
    imagewidth = cor_tomo.shape[2]
    cor_shift_initial_value = 2*(recon_dictionary['cor'] - imagewidth/2)
    non = lambda s: s if s<0 else None
    mom = lambda s: max(0,s)

    fig, ax = plt.subplots(constrained_layout=True)
    fig.canvas.toolbar_position = 'right'
    fig.canvas.header_visible = False
    img = ax.imshow(firstimage-lastimageflipped,cmap='gray',vmin=-0.2,vmax=0.2)
    
    def updateimage(ox):
        recon_dictionary['cor'] = imagewidth/2.0 + ox/2.0
        shiftedlastimage[:,mom(ox):non(ox)] = lastimageflipped[:, mom(-ox):non(-ox)]
        img.set_array(firstimage - shiftedlastimage)
        ax.set_title(f"COR: {recon_dictionary['cor']}")
        fig.canvas.draw()
        fig.canvas.flush_events()
    
    slider_xshift = widgets.IntSlider(description='Shift Image', min = -300, max = 300, step=1, value = cor_shift_initial_value, layout=widgets.Layout(width='50%'))
    uishiftsliders = widgets.HBox([slider_xshift])
    outshiftsliders = widgets.interactive_output(updateimage,{'ox':slider_xshift});
    display(uishiftsliders,outshiftsliders)
else:
    print("COR was already set by user, no display available")

### Given the settings chosen above, the following cell will reconstruct one slice (or the number specificed with `sinoused`)

In [ ]:
start = time.time()

print(f"COR: {recon_dictionary['cor']}")

recon_dictionary["recon_algorithm"] = 'sirt'
recon_dictionary["accelerated"] = True
recon_dictionary["interpolation"] = "NN"
recon_dictionary["device"] = "gpu"
recon_dictionary["pool_size"] = 32
recon_dictionary["ncore"] = 4
recon_dictionary["verbose_printing"] = False
recon_dictionary["iterations"] = 10
recon_dictionary["sinoused"] = (500,510,1)
rec, tomo = recon(**recon_dictionary);

stop = time.time()
print("\nTotal wall time in minutes: ", (stop - start)/60)

### View the small reconstruction

In [ ]:
imagetoshow = rec

number_of_slices = imagetoshow.shape[0]

def showstack(slicenumber):
    plt.figure(figsize=(8, 8))
    plt.imshow(imagetoshow[slicenumber,:,:],cmap='gray',vmin=-0,vmax=255)
    return slicenumber

w = widgets.interactive(showstack, slicenumber = (0,number_of_slices-1,1))
w.children[0].value = 0    
display(w)

### Ready to submit a bigger job to the NERSC Cluster? If all the settings are the same, just run the next cell to prepare ALL sinograms for reconstrution.

Make sure you have changed the key.json file. Please do not share keys.

In [ ]:
functioninput['sinoused'] = None
functioninput['writereconstruction'] = 1
functioninput['verbose_printing'] = 1
functioninput['cor'] = recon_dictionary['cor']
recon_dictionary, cor_tomo = recon_setup(**functioninput)
jwt = get_jwt("key.json")
submission = submit(recon_dictionary, jwt)


### Want to see how the jobs are doing? Copy and paste the following into a terminal (logged into`perlmutter-p1.nersc.gov`): 

`squeue -u <NERSC USERNAME>`